<p style="text-align: center;"><img alt="school-logo" src="../images/school_logo.png" style="zoom: 50%;" /></p>

<h1 align="center">本科生《深度学习》课程<br>实验报告</h1>
<div style="text-align: center;">
    <div><span style="display: inline-block; width: 65px; text-align: center;">课程名称</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">深度学习</span></div>
    <div><span style="display: inline-block; width: 65px; text-align: center;">实验题目</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">Pytorch基本操作</span></div>
    <div><span style="display: inline-block; width: 65px; text-align: center;">学号</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">21281280</span></div>
    <div><span style="display: inline-block; width: 65px; text-align: center;">姓名</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">柯劲帆</span></div>
    <div><span style="display: inline-block; width: 65px; text-align: center;">班级</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">物联网2101班</span></div>
    <div><span style="display: inline-block; width: 65px; text-align: center;">指导老师</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">张淳杰</span></div>
    <div><span style="display: inline-block; width: 65px; text-align: center;">报告日期</span><span style="display: inline-block; width: 25px;">:</span><span style="display: inline-block; width: 210px; font-weight: bold; text-align: left;">2023年10月9日</span></div>
</div>

实验环境：
- OS：Ubuntu 22.04 (Kernel: 6.2.0-34-generic)
- CPU：12th Gen Intel(R) Core(TM) i7-12700H
- GPU：NVIDIA GeForce RTX 3070 Ti Laptop
- cuda: 12.2
- conda: miniconda 23.9.0
- python：3.10.13
- pytorch：2.1.0

In [1]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import datasets, transforms

引用相关库。

# 一、Pytorch基本操作考察
## 题目2
**使用 𝐓𝐞𝐧𝐬𝐨𝐫 初始化一个 𝟏×𝟑 的矩阵 𝑴 和一个 𝟐×𝟏 的矩阵 𝑵，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么。**

In [2]:
A = torch.tensor([[1, 2, 3]])

B = torch.tensor([[4],
                  [5]])

# 方法1: 使用PyTorch的减法操作符
result1 = A - B

# 方法2: 使用PyTorch的sub函数
result2 = torch.sub(A, B)

# 方法3: 手动实现广播机制并作差
def my_sub(a:torch.Tensor, b:torch.Tensor):
    if not ((a.size(0) == 1 and b.size(1) == 1) or (a.size(1) == 1 and b.size(0) == 1)):
        raise ValueError("输入的张量大小无法满足广播机制的条件。")
    else:
        target_shape = torch.Size([max(A.size(0), B.size(0)), max(A.size(1), B.size(1))])
        A_broadcasted = A.expand(target_shape)
        B_broadcasted = B.expand(target_shape)
        result = torch.zeros(target_shape, dtype=torch.int64).to(device=A_broadcasted.device)
        for i in range(target_shape[0]):
            for j in range(target_shape[1]):
                result[i, j] = A_broadcasted[i, j] - B_broadcasted[i, j]
        return result

result3 = my_sub(A, B)

print("方法1的结果:")
print(result1)
print("方法2的结果:")
print(result2)
print("方法3的结果:")
print(result3)

方法1的结果:
tensor([[-3, -2, -1],
        [-4, -3, -2]])
方法2的结果:
tensor([[-3, -2, -1],
        [-4, -3, -2]])
方法3的结果:
tensor([[-3, -2, -1],
        [-4, -3, -2]])


第1、2、3种减法形式实质是一样的。

步骤如下：
1. 对A、B两个张量进行广播，将A、B向广播的方向复制，得到两个$\max(A.size(0), B.size(0))\times \max(A.size(1), B.size(1))$的张量；
2. 对广播后的两个张量作差，尺寸不变。

第1种减法形式和第2种是等价的，前者是后者的符号化表示。

第3种形式是手动实现的，将上述两个步骤分别手动实现了。但是torch.Tensor还内置了其他机制，这里仅模拟了广播和作差。

## 题目2
1. **利用Tensor创建两个大小分别$3\times 2$和$4\times 2$的随机数矩阵$P$和$Q$，要求服从均值为$0$，标准差$0.01$为的正态分布；**
2. **对第二步得到的矩阵$Q$进行形状变换得到$Q$的转置$Q^T$；**
3. **对上述得到的矩阵$P$和矩阵$Q^T$求矩阵相乘。**

In [3]:
mean = 0
stddev = 0.01

P = torch.normal(mean=mean, std=stddev, size=(3, 2))
Q = torch.normal(mean=mean, std=stddev, size=(4, 2))

print("矩阵 P:")
print(P)
print("矩阵 Q:")
print(Q)

# 对矩阵Q进行转置操作，得到矩阵Q的转置Q^T
QT = Q.T
print("矩阵 QT:")
print(QT)

# 计算矩阵P和矩阵Q^T的矩阵相乘
result = torch.matmul(P, QT)
print("矩阵相乘的结果:")
print(result)

矩阵 P:
tensor([[ 0.0069,  0.0082],
        [-0.0052, -0.0124],
        [ 0.0055, -0.0014]])
矩阵 Q:
tensor([[ 0.0050,  0.0075],
        [ 0.0161,  0.0070],
        [-0.0009, -0.0014],
        [-0.0003,  0.0024]])
矩阵 QT:
tensor([[ 0.0050,  0.0161, -0.0009, -0.0003],
        [ 0.0075,  0.0070, -0.0014,  0.0024]])
矩阵相乘的结果:
tensor([[ 9.6016e-05,  1.6860e-04, -1.7451e-05,  1.8011e-05],
        [-1.1894e-04, -1.7065e-04,  2.1900e-05, -2.8712e-05],
        [ 1.6918e-05,  7.8455e-05, -2.7165e-06, -4.9904e-06]])


## 题目2
**给定公式$ y_3=y_1+y_2=𝑥^2+𝑥^3$，且$x=1$。利用学习所得到的Tensor的相关知识，求$y_3$对$x$的梯度，即$\frac{dy_3}{dx}$。**

In [4]:
x = torch.tensor(1.0, requires_grad=True)

y_1 = x ** 2
with torch.no_grad():
    y_2 = x ** 3
y_3 = y_1 + y_2
y_3.backward()
print("仅通过y_1传递的梯度: ", x.grad.item())

x.grad.data.zero_()
with torch.no_grad():
    y_1 = x ** 2
y_2 = x ** 3
y_3 = y_1 + y_2
y_3.backward()
print("仅通过y_2传递的梯度: ", x.grad.item())

x.grad.data.zero_()
y_1 = x ** 2
y_2 = x ** 3
y_3 = y_1 + y_2
y_3.backward()

print("dy_3/dx: ", x.grad.item())

仅通过y_1传递的梯度:  2.0
仅通过y_2传递的梯度:  3.0
dy_3/dx:  5.0


# 二、动手实现logistic回归
## 题目1
**要求动手从0实现 logistic 回归（只借助Tensor和Numpy相关的库）在人工构造的数据集上进行训练和测试，并从loss以及训练集上的准确率等多个角度对结果进行分析（可借助nn.BCELoss或nn.BCEWithLogitsLoss作为损失函数，从零实现二元交叉熵为选作）**

给定预测概率$ \left( \hat{y} \right) $和目标标签$ \left( y \right)$（通常是0或1），BCELoss的计算公式如下：
$$
 \text{BCELoss}(\hat{y}, y) = -\frac{1}{N} \sum_{i=1}^{N} \left(y_i \cdot \log(\hat{y}_i) + (1 - y_i) \cdot \log(1 - \hat{y}_i)\right) 
$$
其中，$\left( N \right) $是样本数量，$\left( \hat{y}_i \right) $表示模型的预测概率向量中的第$ \left( i \right) $个元素，$\left( y_i \right) $表示实际的目标标签中的第$ \left( i \right) $个元素。在二分类问题中，$\left( y_i \right) $通常是0或1。这个公式表示对所有样本的二分类交叉熵损失进行了求和并取平均。

因此BCELoss的手动实现如下。

In [5]:
class My_BCELoss:
    def __call__(self, prediction: torch.Tensor, target: torch.Tensor):
        loss = -torch.mean(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))
        return loss


# 测试
prediction = torch.sigmoid(torch.tensor([0.8]))
target = torch.tensor([1.0])
print(f"输入：\n{prediction}")
print(f"标签：\n{target}")

my_bce_loss = My_BCELoss()
my_loss = my_bce_loss(prediction, target)
print("My_BCELoss损失值:", my_loss.item())

nn_bce_loss = nn.BCELoss()
nn_loss = nn_bce_loss(prediction, target)
print("nn.BCELoss损失值:", nn_loss.item())

输入：
tensor([0.6900])
标签：
tensor([1.])
My_BCELoss损失值: 0.37110066413879395
nn.BCELoss损失值: 0.37110066413879395


Optimizer的实现较为简单。

主要实现：
- 传入参数：`__init__()`
- 对传入的参数进行更新：`step()`
- 清空传入参数存储的梯度：`zero_grad()`

但是有一点需要注意，就是需要将传进来的`params`参数转化为`list`类型。因为`nn.Module`的`parameters()`方法会以`<class 'generator'>`的类型返回模型的参数，但是该类型变量无法像`list`一样使用`for`循环遍历。

In [6]:
class My_Optimizer:
    def __init__(self, params: list[torch.Tensor], lr: float):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for param in self.params:
            param.data = param.data - self.lr * param.grad.data

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.data.zero_()


# 测试
x = torch.tensor(1.0, requires_grad=True)
print("x的初始值: ", x.item())

optimizer_test = My_Optimizer([x], lr=0.1)
print("学习率: ", optimizer_test.lr)

y = x ** 2
y.backward()
print("y.backward()之后，x的梯度: ", x.grad.item())

optimizer_test.step()
print("optimizer_test.step()之后，x的值: ", x.item())

optimizer_test.zero_grad()
print("optimizer_test.zero_grad()之后，x的梯度: ", x.grad.item())

x的初始值:  1.0
学习率:  0.1
y.backward()之后，x的梯度:  2.0
optimizer_test.step()之后，x的值:  0.800000011920929
optimizer_test.zero_grad()之后，x的梯度:  0.0


线性层主要有一个权重（weight)和一个偏置（bias）。
线性层的数学公式如下：
$$
x:=x \times weight^T+bias
$$
因此代码实现如下：

In [7]:
class My_Linear:
    def __init__(self, input_feature: int, output_feature: int):
        self.weight = torch.randn((output_feature, input_feature), requires_grad=True, dtype=torch.float32)
        self.bias = torch.randn(1, requires_grad=True, dtype=torch.float32)
        self.params = [self.weight, self.bias]

    def __call__(self, x: torch.Tensor):
        return self.forward(x)

    def forward(self, x: torch.Tensor):
        x = torch.matmul(x, self.weight.T) + self.bias
        return x

    def to(self, device: str):
        for param in self.params:
            param.data = param.data.to(device=device)
        return self

    def parameters(self):
        return self.params

        
# 测试
my_linear = My_Linear(1, 3)
nn_linear = nn.Linear(1, 3)
my_linear.weight = nn_linear.weight.clone().requires_grad_()
my_linear.bias = nn_linear.bias.clone().requires_grad_()
x = torch.tensor([[1.], [2.]], requires_grad=True)
print(f"输入：\n{x}")
print(f"权重：\n{my_linear.weight.data}")
print(f"偏置：\n{my_linear.bias.data}")
y_my_linear = my_linear(x)
print(f"My_Linear输出：\n{y_my_linear}")
y_nn_linear = nn_linear(x)
print(f"nn.Linear输出：\n{y_nn_linear}")

输入：
tensor([[1.],
        [2.]], requires_grad=True)
权重：
tensor([[ 0.4240],
        [-0.2577],
        [ 0.4972]])
偏置：
tensor([0.6298, 0.6243, 0.8217])
My_Linear输出：
tensor([[1.0539, 0.3666, 1.3189],
        [1.4779, 0.1089, 1.8161]], grad_fn=<AddBackward0>)
nn.Linear输出：
tensor([[1.0539, 0.3666, 1.3189],
        [1.4779, 0.1089, 1.8161]], grad_fn=<AddmmBackward0>)


手动实现logistic回归模型。

模型很简单，主要由一个线性层和一个sigmoid层组成。

Sigmoid函数（又称为 Logistic函数）是一种常用的激活函数，通常用于神经网络的输出层或隐藏层，其作用是将输入的实数值压缩到一个范围在0和1之间的数值。

In [8]:
class Model_2_1():
    def __init__(self):
        self.linear = My_Linear(1, 1)
        self.params = self.linear.params

    def __call__(self, x):
        return self.forward(x)

    def forward(self, x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x

    def to(self, device: str):
        for param in self.params:
            param.data = param.data.to(device=device)
        return self

    def parameters(self):
        return self.params

人工随机构造数据集。

这里我遇到了比较大的问题。因为数据构建不合适，会导致后面的训练出现梯度爆炸。

我采用随机产生数据后归一化的方法，即
$$
\hat{x} = \frac{x - \text{min}_x}{\text{max}_x - \text{min}_x} 
$$
将数据控制在合适的区间。

我的y设置为$4-3\times x + noise$，noise为随机噪声。

生成完x和y后进行归一化处理，并写好DataLoader访问数据集的接口`__getitem__()`。

In [9]:
class My_Dataset(Dataset):
    def __init__(self, data_size=1000000):
        np.random.seed(0)
        x = 2 * np.random.rand(data_size, 1)
        noise = 0.2 * np.random.randn(data_size, 1)
        y = 4 - 3 * x + noise
        self.min_x, self.max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        x = (x - self.min_x) / (self.max_x - self.min_x)
        y = (y - min_y) / (max_y - min_y)
        self.data = [[x[i][0], y[i][0]] for i in range(x.shape[0])]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x, y = self.data[index]
        return x, y


# 测试，并后面的训练创建变量
dataset = My_Dataset()
dataset_size = len(dataset)
print(f"测试数据集大小：{dataset_size}")
x0, y0 = dataset[0]
print(f"测试数据集第0对数据：")
print(f"x_0 = {x0}")
print(f"y_0 = {y0}")

测试数据集大小：1000000
测试数据集第0对数据：
x_0 = 0.5488133381316141
y_0 = 0.45217091576438073


训练Logistic回归模型。
进行如下步骤：
1. 初始化超参数
2. 获取数据集
3. 初始化模型
4. 定义损失函数和优化器
5. 训练
    1. 从训练dataloader中获取批量数据
    2. 传入模型
    3. 使用损失函数计算与ground_truth的损失
    4. 使用优化器进行反向传播
    5. 循环以上步骤
6. 测试
    1. 设置测试数据
    2. 传入模型
    3. 得到预测值

In [10]:
learning_rate = 5e-2
num_epochs = 10
batch_size = 1024
device = "cuda:0" if torch.cuda.is_available() else "cpu"

dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=14, pin_memory=True)

model = Model_2_1().to(device)
criterion = My_BCELoss()
optimizer = My_Optimizer(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_epoch_loss = 0
    total_epoch_pred = 0
    total_epoch_target = 0
    for x, targets in dataloader:
        optimizer.zero_grad()
        
        x = x.to(device).to(dtype=torch.float32)
        targets = targets.to(device).to(dtype=torch.float32)
        
        x = x.unsqueeze(1)
        y_pred = model(x)
        loss = criterion(y_pred, targets)
        total_epoch_loss += loss.item()
        total_epoch_target += targets.sum().item()
        total_epoch_pred += y_pred.sum().item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_epoch_loss}, ", end="")
    print(f"Acc: {1 - abs(total_epoch_pred - total_epoch_target) / total_epoch_target}")

with torch.no_grad():
    test_data = (np.array([[2]]) - dataset.min_x) / (dataset.max_x - dataset.min_x)
    test_data = Variable(torch.tensor(test_data, dtype=torch.float32), requires_grad=False).to(device)
    predicted = model(test_data).to("cpu")
    print(f"Model weights: {model.linear.weight.item()}, bias: {model.linear.bias.item()}")
    print(f"Prediction for test data: {predicted.item()}")

Epoch 1/10, Loss: 688.6783249974251, Acc: 0.9766838179955138
Epoch 2/10, Loss: 679.506599009037, Acc: 0.992039453911494
Epoch 3/10, Loss: 677.644762635231, Acc: 0.9961844975781526
Epoch 4/10, Loss: 677.2690716981888, Acc: 0.998395304269398
Epoch 5/10, Loss: 677.1928514242172, Acc: 0.9993592246184307
Epoch 6/10, Loss: 677.1781670451164, Acc: 0.9996570376204033
Epoch 7/10, Loss: 677.1744618415833, Acc: 0.9998465339227576
Epoch 8/10, Loss: 677.1738814711571, Acc: 0.9998001679325041
Epoch 9/10, Loss: 677.1742851734161, Acc: 0.9998804348705138
Epoch 10/10, Loss: 677.1740592718124, Acc: 0.9999446971149187
Model weights: -0.0037125118542462587, bias: 0.017451055347919464
Prediction for test data: 0.5034345984458923


## 题目2
**利用 torch.nn 实现 logistic 回归在人工构造的数据集上进行训练和测试，并对结果进行分析，并从loss以及训练集上的准确率等多个角度对结果进行分析**

使用torch.nn实现模型。

将之前的Model_2_1中的手动实现函数改为torch.nn内置函数即可，再加上继承nn.Module以使用torch.nn内置模型模板特性。

In [11]:
class Model_2_2(nn.Module):
    def __init__(self):
        super(Model_2_2, self).__init__()
        self.linear = nn.Linear(1, 1, dtype=torch.float64)

    def forward(self, x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x

训练与测试过程与之前手动实现的几乎一致。仅有少量涉及数据类型（dtype）的代码需要更改以适应torch.nn的内置函数要求。

In [12]:
learning_rate = 5e-2
num_epochs = 10
batch_size = 1024
device = "cuda:0" if torch.cuda.is_available() else "cpu"

dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=14, pin_memory=True)

model = Model_2_2().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_epoch_loss = 0
    total_epoch_pred = 0
    total_epoch_target = 0
    for x, targets in dataloader:
        optimizer.zero_grad()

        x = x.to(device)
        targets = targets.to(device)

        x = x.unsqueeze(1)
        targets = targets.unsqueeze(1)
        y_pred = model(x)
        loss = criterion(y_pred, targets)
        total_epoch_loss += loss.item()
        total_epoch_target += targets.sum().item()
        total_epoch_pred += y_pred.sum().item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_epoch_loss}, ", end="")
    print(f"Acc: {1 - abs(total_epoch_pred - total_epoch_target) / total_epoch_target}")

with torch.no_grad():
    test_data = (np.array([[2]]) - dataset.min_x) / (dataset.max_x - dataset.min_x)
    test_data = Variable(torch.tensor(test_data, dtype=torch.float64), requires_grad=False).to(device)
    predicted = model(test_data).to("cpu")
    print(f"Model weights: {model.linear.weight.item()}, bias: {model.linear.bias.item()}")
    print(f"Prediction for test data: {predicted.item()}")

Epoch 1/10, Loss: 660.2008021697803, Acc: 0.9355364605682331
Epoch 2/10, Loss: 589.2025169091534, Acc: 0.9769773185253259
Epoch 3/10, Loss: 572.7106042209589, Acc: 0.9881629137259633
Epoch 4/10, Loss: 568.0903503441508, Acc: 0.9935173218188225
Epoch 5/10, Loss: 566.6528526848851, Acc: 0.9962586560919562
Epoch 6/10, Loss: 566.1778871576632, Acc: 0.9978209774304773
Epoch 7/10, Loss: 566.0143385848835, Acc: 0.9987369762885633
Epoch 8/10, Loss: 565.9605239629793, Acc: 0.9992563563084009
Epoch 9/10, Loss: 565.9402079010808, Acc: 0.9995321069396558
Epoch 10/10, Loss: 565.9281422200424, Acc: 0.9997496312356398
Model weights: -3.6833968323036084, bias: 1.8628376037952126
Prediction for test data: 0.13936666014014443


对比发现，手动实现的损失函数和优化器与torch.nn的内置损失函数和优化器相比，表现差不多。

但是为什么相同分布的数据集训练出的权重和偏置，以及预测结果存在较大差别，这个问题的原因还有待我探究。

# 三、动手实现softmax回归

## 问题1

**要求动手从0实现softmax回归（只借助Tensor和Numpy相关的库）在Fashion-MNIST数据集上进行训练和测试，并从loss、训练集以及测试集上的准确率等多个角度对结果进行分析（要求从零实现交叉熵损失函数）**

手动实现nn.one_hot()。

one-hot向量用于消除线性标签值所映射的类别的非线性。

one-hot向量是使用一个长度为分类数量的数组表示标签值，其中有且仅有1个值为为1，该值的下标为标签值；其余为0。

原理很简单，步骤如下：
1. 初始化全零的张量，大小为（标签数量，分类数量）；
2. 将标签值映射到全零张量的\[下标，标签值\]中，将该位置为1；
3. 返回修改后的张量，即是ont-hot向量。

In [13]:
def my_one_hot(indices: torch.Tensor, num_classes: int):
    one_hot_tensor = torch.zeros(len(indices), num_classes).to(indices.device).to(dtype=torch.int64)
    one_hot_tensor.scatter_(1, indices.view(-1, 1), 1)
    return one_hot_tensor


# 测试
x = torch.tensor([2, 1, 0], dtype=torch.int64)
print(f"输入：\n{x}")

x_my_onehot = my_one_hot(x, 5)
print(f"my_one_hot输出：\n{x_my_onehot}")

x_nn_F_onehot = nn.functional.one_hot(x, 5)
print(f"nn.functional.one_hot输出：\n{x_nn_F_onehot}")

输入：
tensor([2, 1, 0])
my_one_hot输出：
tensor([[0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0]])
nn.functional.one_hot输出：
tensor([[0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0]])


手动实现CrossEntropyLoss。

CrossEntropyLoss由一个log_softmax和一个nll_loss组成。

softmax的数学表达式如下：
$$
\text{softmax}(y_i) = \frac{e^{y_i - \text{max}(y)}}{\sum_{j=1}^{N} e^{y_j - \text{max}(y)}} 
$$
log_softmax即为$\log\left(softmax\left(y\right)\right)$。

CrossEntropyLoss的数学表达式如下：
$$
\text{CrossEntropyLoss}(y, \hat{y}) = -\frac{1}{N} \sum_{i=1}^{N} \hat{y}_i \cdot \log(\text{softmax}(y_i)) 
$$

故代码如下：

In [14]:
class My_CrossEntropyLoss:
    def __call__(self, predictions: torch.Tensor, targets: torch.Tensor):
        max_values = torch.max(predictions, dim=1, keepdim=True).values
        exp_values = torch.exp(predictions - max_values)
        softmax_output = exp_values / torch.sum(exp_values, dim=1, keepdim=True)
        log_probs = torch.log(softmax_output)
        
        nll_loss = -torch.sum(targets * log_probs, dim=1)
        average_loss = torch.mean(nll_loss)
        return average_loss

        
# 测试
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1).argmax(1)
target = torch.nn.functional.one_hot(target, num_classes=5).to(dtype=torch.float32)
print(f"输入：\n{input}")
print(f"标签：\n{target}")

my_crossentropyloss = My_CrossEntropyLoss()
my_loss = my_crossentropyloss(input, target)
print("My_CrossEntropyLoss损失值:", my_loss.item())

nn_crossentropyloss = nn.CrossEntropyLoss()
nn_loss = nn_crossentropyloss(input, target)
print("nn.CrossEntropyLoss损失值:", nn_loss.item())

输入：
tensor([[ 0.7600,  0.4269,  0.7948, -0.6086,  1.2527],
        [-0.4749,  0.5720, -0.0164, -0.2126, -0.0410],
        [ 1.3269,  1.8524, -0.9815,  0.0156,  1.6971]], requires_grad=True)
标签：
tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])
My_CrossEntropyLoss损失值: 1.7417106628417969
nn.CrossEntropyLoss损失值: 1.7417105436325073


手动实现Flatten。

原理很简单，就是把多维的张量拉直成一个向量。

In [15]:
class My_Flatten:
    def __call__(self, x: torch.Tensor):
        return self.forward(x)

    def forward(self, x: torch.Tensor):
        x = x.view(x.shape[0], -1)
        return x


# 测试
my_flatten = My_Flatten()
nn_flatten = nn.Flatten()
x = torch.tensor([[[1., 2.],
                   [3., 4.]],
                  [[5., 6.],
                   [7., 8.]]])
print(f"Flatten之前的x：\n{x}")
x_my_flatten = my_flatten(x)
print(f"My_Flatten之后的x：\n{x_my_flatten}")
x_nn_flatten = nn_flatten(x)
print(f"nn.Flatten之后的x：\n{x_nn_flatten}")

Flatten之前的x：
tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])
My_Flatten之后的x：
tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])
nn.Flatten之后的x：
tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])


手动实现softmax回归模型。

模型很简单，主要由一个Flatten层和一个线性层组成。

Flatten层主要用于将2维的图像展开，直接作为1维的特征量输入网络。

In [16]:
class Model_3_1:
    def __init__(self, num_classes):
        self.flatten = My_Flatten()
        self.linear = My_Linear(28 * 28, num_classes)
        self.params = self.linear.params

    def __call__(self, x: torch.Tensor):
        return self.forward(x)

    def forward(self, x: torch.Tensor):
        x = self.flatten(x)
        x = self.linear(x)
        return x

    def to(self, device: str):
        for param in self.params:
            param.data = param.data.to(device=device)
        return self

    def parameters(self):
        return self.params

训练与测试过程与之前手动实现的几乎一致。由于数据集的变化，对应超参数也进行了调整。

数据集也使用了现成的FashionMNIST数据集，且划分了训练集和测试集。

FashionMNIST数据集直接调用API获取。数据集的image为28*28的单通道灰白图片，label为单个数值标签。

训练softmax回归模型。
进行如下步骤：
1. 初始化超参数
2. 获取数据集
3. 初始化模型
4. 定义损失函数和优化器
5. 训练
    1. 从训练dataloader中获取批量数据
    2. 传入模型
    3. 使用损失函数计算与ground_truth的损失
    4. 使用优化器进行反向传播
    5. 循环以上步骤
6. 测试
    1. 从测试dataloader中获取批量数据
    2. 传入模型
    3. 将预测值与ground_truth进行比较，得出正确率
    4. 对整个训练集统计正确率，从而分析训练效果

In [17]:
learning_rate = 5e-1
num_epochs = 10
batch_size = 4096
num_classes = 10
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (1.0,)),
    ]
)
train_dataset = datasets.FashionMNIST(root="./dataset", train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root="./dataset", train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True, num_workers=14, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=True, num_workers=14, pin_memory=True)

model = Model_3_1(num_classes).to(device)
criterion = My_CrossEntropyLoss()
optimizer = My_Optimizer(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_epoch_loss = 0
    for images, targets in train_loader:
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device).to(dtype=torch.long)

        one_hot_targets = my_one_hot(targets, num_classes=num_classes).to(device).to(dtype=torch.long)

        outputs = model(images)
        loss = criterion(outputs, one_hot_targets)
        total_epoch_loss += loss

        loss.backward()
        optimizer.step()

    total_acc = 0
    with torch.no_grad():
        for image, targets in test_loader:
            image = image.to(device)
            targets = targets.to(device)
            outputs = model(image)
            total_acc += (outputs.argmax(1) == targets).sum()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_epoch_loss}, Acc: {total_acc / len(test_dataset)}")

Epoch 1/10, Loss: 87.64246368408203, Acc: 0.45329999923706055
Epoch 2/10, Loss: 42.025726318359375, Acc: 0.5523999929428101
Epoch 3/10, Loss: 34.06425094604492, Acc: 0.5947999954223633
Epoch 4/10, Loss: 30.135021209716797, Acc: 0.620199978351593
Epoch 5/10, Loss: 27.43822479248047, Acc: 0.6401000022888184
Epoch 6/10, Loss: 25.72039031982422, Acc: 0.6525999903678894
Epoch 7/10, Loss: 24.28335952758789, Acc: 0.6638999581336975
Epoch 8/10, Loss: 23.18214988708496, Acc: 0.671999990940094
Epoch 9/10, Loss: 22.18520164489746, Acc: 0.680899977684021
Epoch 10/10, Loss: 21.393451690673828, Acc: 0.6875999569892883


在这里我遇到了梯度爆炸的问题。

原来我在数据预处理中使用`transforms.Normalize((0.5,), (0.5,))`进行归一化，但是这样导致了梯度爆炸。

将第二个参数方差改为1.0后，成功解决了梯度爆炸的问题。

## 问题2

**利用torch.nn实现softmax回归在Fashion-MNIST数据集上进行训练和测试，并从loss，训练集以及测试集上的准确率等多个角度对结果进行分析**

使用torch.nn实现模型。

将之前的Model_3_1中的手动实现函数改为torch.nn内置函数即可，再加上继承nn.Module以使用torch.nn内置模型模板特性。

In [18]:
class Model_3_2(nn.Module):
    def __init__(self, num_classes):
        super(Model_3_2, self).__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(28 * 28, num_classes)

    def forward(self, x: torch.Tensor):
        x = self.flatten(x)
        x = self.linear(x)
        return x

训练与测试过程与之前手动实现的几乎一致。

In [19]:
learning_rate = 5e-2
num_epochs = 10
batch_size = 4096
num_classes = 10
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)
train_dataset = datasets.FashionMNIST(root="./dataset", train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root="./dataset", train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=14, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, num_workers=14, pin_memory=True)

model = Model_3_2(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_epoch_loss = 0
    model.train()
    for images, targets in train_loader:
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        one_hot_targets = nn.functional.one_hot(targets, num_classes=num_classes).to(device).to(dtype=torch.float32)

        outputs = model(images)
        loss = criterion(outputs, one_hot_targets)
        total_epoch_loss += loss

        loss.backward()
        optimizer.step()

    model.eval()
    total_acc = 0
    with torch.no_grad():
        for image, targets in test_loader:
            image = image.to(device)
            targets = targets.to(device)
            outputs = model(image)
            total_acc += (outputs.argmax(1) == targets).sum()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_epoch_loss}, Acc: {total_acc / len(test_dataset)}")

Epoch 1/10, Loss: 19.15451431274414, Acc: 0.7202000021934509
Epoch 2/10, Loss: 12.260371208190918, Acc: 0.7486000061035156
Epoch 3/10, Loss: 10.835549354553223, Acc: 0.7615999579429626
Epoch 4/10, Loss: 10.09542179107666, Acc: 0.7701999545097351
Epoch 5/10, Loss: 9.626176834106445, Acc: 0.777899980545044
Epoch 6/10, Loss: 9.264442443847656, Acc: 0.7854999899864197
Epoch 7/10, Loss: 9.017412185668945, Acc: 0.7879999876022339
Epoch 8/10, Loss: 8.786051750183105, Acc: 0.7915999889373779
Epoch 9/10, Loss: 8.613431930541992, Acc: 0.79749995470047
Epoch 10/10, Loss: 8.462657928466797, Acc: 0.7996999621391296


与手动实现的softmax回归相比较，nn.CrossEntropyLoss比手动实现的My_CrossEntropyLoss更加稳定，对输入数据的兼容性更强，没有出现梯度爆炸的情况。

总体表现上，torch.nn的内置功能相对手动实现的功能，正确率提升更快，最终正确率更高。

# 实验心得体会

通过完成本次Pytorch基本操作实验，让我对Pytorch框架有了更加深入的理解。我接触深度学习主要是在大语言模型领域，比较熟悉微调大模型，但是涉及到底层的深度学习知识，我还有很多短板和不足。这次实验对我这方面的锻炼让我收获良多。

首先是数据集的设置。如果数据没有合理进行归一化，很容易出现梯度爆炸。这是在我以前直接使用图片数据集的经历中没有遇到过的问题。

在实现logistic回归模型时，通过手动实现各个组件如优化器、线性层等，让我对这些模块的工作原理有了更清晰的认识。尤其是在实现广播机制时，需要充分理解张量操作的维度变换规律。而使用Pytorch内置模块进行实现时，通过继承nn.Module可以自动获得许多功能，使代码更加简洁。

在实现softmax回归时，则遇到了更大的困难。手动实现的模型很容易出现梯度爆炸的问题，而使用Pytorch内置的损失函数和优化器则可以稳定训练。这让我意识到了选择合适的优化方法的重要性。另外，Pytorch强大的自动微分机制也是构建深度神经网络的重要基础。

通过这个实验，让我对Pytorch框架有了更加直观的感受，也让我看到了仅靠基础模块搭建复杂模型的难点所在。这些经验对我后续使用Pytorch构建数据集模型会很有帮助。